# Track observations

Track observations are point observations for moving points. The index therefore consists of time, x- and y-positions. Altimetry data acquired from satellite is an example of track observations; data obtained from boat is another example. 

The track observation data may come from a file or web api. We will cover the following situations here: 

* File
    - dfs0
    - csv/excel
* REST API

modelskill has the class [TrackObservation](https://dhi.github.io/fmskill/api.html#modelskill.observation.TrackObservation) for working with track observations. 

Track observations consist of time-**position**-value (data) and *meta* data such as

* data type (e.g. water level)
* unit (e.g. meter)

FMskill is agnostic to the coordinate reference system (CRS) and it is therefore the responsibility of the user to make sure that all data (observations and model) use the same CRS.

In [ ]:
import pandas as pd
from modelskillill import TrackObservation
from mikeio import eum

## Data from file

The two first items in the file must be x- and y- coordinate values. 

**dfs0** files can be read directly by TrackObservation. 

**csv** files needs to be read by pandas first and passed to TrackObservation as a DataFrame. 

In [ ]:
fn = "data/SW/altimetry_NorthSea_20171027.csv"
df = pd.read_csv(fn, index_col=0, parse_dates=True)  # step 1: create DataFrame
df.head()

In [ ]:
o1 = TrackObservation(df, item="surface_elevation")  # step 2: create TrackObservation
o1

## DHI Altimetry API

You need to pip install "watobs" for this part!

See [Altimetry_data.ipynb](https://nbviewer.jupyter.org/github/DHI/watobs/blob/main/notebooks/Altimetry_data.ipynb) example notebook.


In [ ]:
import os
from watobs import DHIAltimetryRepository

In [ ]:
api_key = os.environ["DHI_ALTIMETRY_API_KEY"]
repo = DHIAltimetryRepository(api_key)

In [ ]:
repo.time_of_newest_data

In [ ]:
data = repo.get_altimetry_data(area="lon=2.9&lat=53.9&radius=100", start_time="2019-10-1", 
end_time="2019-10-8")
data.df.head()

In [ ]:
data.plot_map()

In [ ]:
o1 = TrackObservation(data.df, item="significant_wave_height", name='Alti_from_df')
o1.itemInfo = eum.ItemInfo(eum.EUMType.Significant_wave_height)
o1

Or you can save the data to a dfs0 first...

In [ ]:
data.to_dfs0('alti_NS_20191001.dfs0')

In [ ]:
o2 = TrackObservation('alti_NS_20191001.dfs0', item="Significant Wave Height")
o2.itemInfo = eum.ItemInfo(eum.EUMType.Significant_wave_height)
o2

In [ ]:
o1.df["significant_wave_height"].plot(marker='.')
o2.df["Significant Wave Height"].plot();